# Getting and solving a collection of JGEX problems from a file

Depending on your goals, for example if you want to get statistics of some improvement of Newclid over a benchmark, you may want to run Newclid over a collection of problems contained in a single file. This tutorial teaches you to read a file containing JGEX formulations.

In [ ]:
try:
    import newclid
except ImportError:
    !pip install -q "newclid>=3" "py-yuclid"

## Typical structure of the files and reading files

The traditional structure of a problem file is a .txt file of pairs of lines in the format:

````
    Problem1_name
    Problem2_formulation
    Problem2_name
    Problem2_formulation
    ...
````

The function `jgex_formulation_from_txt_file()` will read the path to your file and turn it into a dictionary of JGEX formulations with their problem names.

Below, the variable `DATASET_PATH` contains the path to the folder with you problems file and `problem_file_name` the name of the file.


In [1]:
from pathlib import Path

from newclid.jgex.formulation import jgex_formulation_from_txt_file

DATASET_PATH = Path("../newclid/problems_datasets")
problem_file_name = "imo_sl.txt"

PROBLEM_SET = jgex_formulation_from_txt_file(DATASET_PATH.joinpath(problem_file_name))

## Building single problem

Now you can build a single problem from the file using its name. Notice that you have to use the `with_problem()` method of the `JGEXProblemBuilder`, because the problem strings were already processed into `JGEXFormulation`s. You only have to pass a name of a problem that is in your file as the dictionary key.

In [2]:
import numpy as np
from newclid.jgex.problem_builder import JGEXProblemBuilder

rng = np.random.default_rng(0)


single_problem_builder = (
    JGEXProblemBuilder(rng)
    .include_auxiliary_clauses(True)
    .with_problem(PROBLEM_SET["imo_sl_2003_g1a"])
)

## Check build

As for the GeoGebra problem, you can build your `JGEXProblemBuilder` into an `NcProblem` and check if it is the problem you are looking for, by extracting its points, hypothesis and goals.

In [3]:
nc_single_problem = single_problem_builder.build()
print(nc_single_problem.pretty_str())

Built the problem problem with assumptions:
coll a c e
diff a c
diff d p
diff d p
diff d q
diff d q
diff d r
diff d r
diff p r
eqangle a b b e b e b c
midp q p r
ncoll a b c
perp a q d q
perp a r d r
perp b p d p
perp b r d r
perp c p d p
perp c q d q

and goals:
eqangle a d d e d e c d

Using points:
a (-3.44663830029546, 1.2119855048584467)
b (-2.3365335504485585, 0.25538706380500314)
c (-1.5401258999603105, 0.037004047875694675)
d (0.6265404784005448, 0.8255111545554434)
e (-2.2272776353069457, 0.46049489274189276)
p (0.2739233746429086, -0.4604265724722594)
q (-0.322064788742351, -0.7136856507076006)
r (-0.9180529521276106, -0.9669447289429418)



## Solving your NcProblem

As we have turned our problem into an NcProblem, we can solve it exactly as we did for GeoGebra problems.

In [4]:
from newclid.api import GeometricSolverBuilder

solver = GeometricSolverBuilder(rng=rng).build(nc_single_problem)
solver.run()

natural_language_proof = solver.proof()
print(natural_language_proof)
print()

assert solver.run_infos is not None, (
    "Solver needs to be run before accessing the run_infos"
)
print("Run infos:\n", solver.run_infos.model_dump_json(indent=2))

# Problem setup:

## Points

- A(-3.4466, 1.2120)
- B(-2.3365, 0.2554)
- C(-1.5401, 0.0370)
- D(0.6265, 0.8255)
- E(-2.2273, 0.4605)
- P(0.2739, -0.4604)
- Q(-0.3221, -0.7137)
- R(-0.9181, -0.9669)

## Assumptions:

[C0] : A, C, E are collinear
[C1] : BP ⟂ DP
[C2] : BR ⟂ DR
[C3] : AR ⟂ DR
[C4] : Q is the midpoint of PR
[C5] : AQ ⟂ DQ
[C6] : ∠(AB,BE) = ∠(BE,BC)
[C7] : CP ⟂ DP
[C8] : CQ ⟂ DQ

## Numerical checks

[N0] : ▲ABD has the same orientation as ▲DQP
[N1] : B, P, R are not collinear
[N2] : A ≠ B
[N3] : A ≠ R
[N4] : B ≠ R
[N5] : P ≠ Q
[N6] : P ≠ R
[N7] : Q ≠ R
[N8] : D, Q, R are not collinear
[N9] : A, B, C are not collinear
[N10] : ▲BCD has the same orientation as ▲DRQ
[N11] : C, P, Q are not collinear
[N12] : B ≠ C
[N13] : B ≠ P
[N14] : C ≠ P
[N15] : A, C, D are not collinear


# Goals

∠(AD,DE) = ∠(DE,CD) : Proved [30]


# Proof:

000. | BP ⟂ DP [C1], BR ⟂ DR [C2] =(AR Deduction)> ∠(BP,BR) = ∠(DP,DR) [0]
001. | ∠(BP,BR) = ∠(DP,DR) [0], B, P, R are not collinear [N1] =(r04 Congru

## Solving the list of problems

You can also want to get statistics into which problems in your list are solved or built. For this, you only need to iterate over the `PROBLEM_SET` dictionary.

In [ ]:
solved_problems = 0
not_solved_problems = 0
built_problems = 0
not_built_problems = 0

for problem_name, problem in PROBLEM_SET.items():
    description_string = problem_name + ": "
    try:
        problem_setup = (
            JGEXProblemBuilder(rng)
            .include_auxiliary_clauses(True)
            .with_problem(problem)
            .build()
        )
        built_problems += 1
        description_string += "Built | "
    except Exception:
        not_built_problems += 1
        description_string += "Not built"
        print(description_string)
        continue
    solver = GeometricSolverBuilder(rng=rng).build(problem_setup)
    success = solver.run()
    if success:
        solved_problems += 1
        description_string += "Solved"
    else:
        not_solved_problems += 1
        description_string += "Not solved"
    print(description_string)

print("\nResults:)")
print(f"{built_problems}/{len(PROBLEM_SET)} built")
print(f"{not_built_problems}/{len(PROBLEM_SET)} not built")
print(f"{solved_problems}/{len(PROBLEM_SET)} solved")
print(f"{not_solved_problems}/{len(PROBLEM_SET)} not solved")

imo_sl_2000_g2: Built | Solved
imo_sl_2000_g6_constr: Built | Solved
imo_sl_2000_g8_constr: Built | Solved
imo_sl_2002_g1: Built | Solved
imo_sl_2002_g3a: Built | Solved
imo_sl_2002_g3b: Built | Solved
imo_sl_2002_g4: Built | Solved
imo_sl_2003_g1a: Built | Solved
imo_sl_2003_g1b: Built | Solved
imo_sl_2003_g5: Built | Solved
imo_sl_2004_g1: Built | Solved
imo_sl_2004_g2: Built | Solved
imo_sl_2004_g3b: Built | Solved
imo_sl_2004_g3c: Built | Solved
imo_sl_2004_g4a: Built | Solved
imo_sl_2005_g2: Built | Solved
imo_sl_2005_g4: Built | Solved
imo_sl_2005_g5: Built | Solved
imo_sl_2006_g2_constr: Built | Solved
imo_sl_2006_g3: Built | Solved
imo_sl_2006_g4_constr: Built | Solved
imo_sl_2006_g6_constr: Built | Solved
imo_sl_2006_g9: Built | Solved
imo_sl_2007_g1: Built | Solved
imo_sl_2007_g2_constr: Built | Solved
imo_sl_2007_g3_constr: Built | Solved
imo_sl_2008_g1a_constr: Built | Solved
imo_sl_2008_g3_constr: Built | Solved
imo_sl_2009_g2_constr: Built | Solved
imo_sl_2009_g4_constr: 